## 通过大模型解析图文

#### 1. PDF 转图片

- 需要提前安装 poppler-utils 包，处理PDF文件所需的 poppler 工具
    ```bash
    sudo apt update && sudo apt install -y poppler-utils
    ```

In [12]:
import os
from pdf2image import convert_from_path

output_dir = "temp_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

images = convert_from_path("../../90-文档_Data/黑神话/黑神话悟空.pdf")
image_paths = []
for i, image in enumerate(images):
    image_path = os.path.join(output_dir, f'page_{i+1}.jpg')
    image.save(image_path, 'JPEG')
    image_paths.append(image_path)
print(f"成功转换 {len(image_paths)} 页")

成功转换 5 页


#### 2. 使用大模型 分析图片

In [13]:
from openai import OpenAI
import base64

# 初始化OpenAI客户端
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
    base_url=os.getenv('OPENAI_API_BASE')
)  

print("\n开始分析图片...")
results = []
for image_path in image_paths:
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "请详细描述这张PPT幻灯片的内容，包括标题、正文和图片内容。"},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        max_tokens=300
    )
results.append(response.choices[0].message.content)
print("\n图片分析完成。")


开始分析图片...

图片分析完成。


#### 3. 转换为 LangChain 的 Document 数据结构

In [14]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content=result,
        metadata={"source": "../../90-文档_Data/黑神话/黑神话悟空.pdf", "page_number": i + 1}
    )
    for i, result in enumerate(results)
]

# 输出所有生成的 Document 对象
print("\n分析结果：")
for doc in documents:
    print(f"内容: {doc.page_content}\n元数据: {doc.metadata}\n")
    print("-" * 80)

# 清理临时文件
for image_path in image_paths:
    os.remove(image_path)
os.rmdir(output_dir)


分析结果：
内容: 这张PPT幻灯片的内容如下：

**标题**: 天命

**正文**: 
- 内容为：“天命人，你，也想当神仙？”表达了一种对命运的思考和对超凡境界的向往。

**图片内容**: 
- 幻灯片中展示了一位神话角色的侧脸，面部特征明显，拥有角状的装饰和长长的白色胡须，皮肤呈现绿色调，表情庄重。背景为深色，与角色的颜色形成对比，使其更加突出。

整体设计风格神秘且具有东方神话色彩。
元数据: {'source': '../../90-文档_Data/黑神话/黑神话悟空.pdf', 'page_number': 1}

--------------------------------------------------------------------------------
